In [ ]:
%cd ..

import ibkr
from analysis import liveValuesForPositions
from ib_insync import IB
from model import Stock
import pandas as pd

ib = IB()
ib.connect('127.0.0.1', port = 4001)
positions = ibkr.downloadPositions(ib, lenient=False)

In [ ]:
stockPositions = [p for p in positions if isinstance(p.instrument, Stock)]
stockPositions

In [ ]:
values = liveValuesForPositions(stockPositions, ibkr.IBDataProvider(ib))

In [ ]:
rows = {p.instrument.symbol: values[p] for p in stockPositions}
pd.DataFrame.from_dict(data=rows, orient='index', columns=['Market value'])